In [1]:
#https://jsdokken.com/dolfinx-tutorial/chapter2/navierstokes.html#variational-formulation

from dolfinx import mesh, fem, io
import ufl
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np

T = 5
num_steps = 1000
n_cells = 32

In [2]:
domain = mesh.create_unit_square(MPI.COMM_WORLD, n_cells, n_cells)
velocity_function_space = fem.functionspace(domain, ("Lagrange", 2, (2,))) 
pressure_function_space = fem.functionspace(domain, ("Lagrange", 1))

In [3]:
f = fem.Constant(domain, PETSc.ScalarType((0,0)))
dt = fem.Constant(domain, T/num_steps)
mu = fem.Constant(domain, PETSc.ScalarType(1))
rho = fem.Constant(domain, PETSc.ScalarType(1))

In [4]:
u = ufl.TrialFunction(velocity_function_space)
p = ufl.TrialFunction(pressure_function_space)

v = ufl.TestFunction(velocity_function_space)
q = ufl.TestFunction(pressure_function_space)

u_sol = fem.Function(velocity_function_space) # function to store u solved
u_prev = fem.Function(velocity_function_space) # u from previous time step
p_sol = fem.Function(pressure_function_space) 
p_prev = fem.Function(pressure_function_space)

In [5]:
# poisellieu flow
def inflow(x):
    return np.isclose(x[0], 0)

def outflow(x):
    return np.isclose(x[0], 1)

def walls(x):
    return np.logical_or(
        np.isclose(x[1], 0), np.isclose(x[1], 1)
   )

fdim = domain.topology.dim - 1
inflow_facets = mesh.locate_entities_boundary(domain, fdim, inflow)
dofs_inflow = fem.locate_dofs_topological(pressure_function_space, fdim, inflow_facets)
bc_inflow  = fem.dirichletbc(fem.Constant(domain, PETSc.ScalarType(8)), dofs_inflow, pressure_function_space)

outflow_facets = mesh.locate_entities_boundary(domain, fdim, outflow)
dofs_outflow = fem.locate_dofs_topological(pressure_function_space, fdim, outflow_facets)
bc_outflow  = fem.dirichletbc(fem.Constant(domain, PETSc.ScalarType(0)), dofs_outflow, pressure_function_space)
bc_p = [bc_inflow, bc_outflow]

walls_facets = mesh.locate_entities_boundary(domain, fdim, walls)
dofs_walls = fem.locate_dofs_topological(velocity_function_space, fdim, walls_facets)
bc_noslip  = fem.dirichletbc(fem.Constant(domain, PETSc.ScalarType((0, 0))), dofs_walls, velocity_function_space)
bc_u = [bc_noslip]

In [6]:
from ufl import FacetNormal, dx, ds, dot, inner, sym, nabla_grad, Identity, lhs, rhs, div

u_midpoint = 0.5*(u_prev + u)
n = FacetNormal(domain)

def epsilon(u):
    return sym(nabla_grad(u))

def sigma(u, p):
    return 2*mu*epsilon(u) - p*Identity(len(u))

# step 1
form1 = rho*dot((u - u_prev) / dt, v)*dx \
      + rho*dot(dot(u_prev, nabla_grad(u_prev)), v)*dx \
      + inner(sigma(u_midpoint, p_prev), epsilon(v))*dx \
      + dot(p_prev*n, v)*ds - dot(mu*nabla_grad(u_midpoint)*n, v)*ds \
      - dot(f, v)*dx
bilinear1 = lhs(form1)
linear1 = rhs(form1)

# bilinear1 is not time dependent, it can be assembled only once
#A1 = fem.assemble_matrix(fem.form(bilinear1), bcs=bc_u)
#A1.assemble()

#linear1 is time dependent
#b1 = create_vector(linear1) # 

# step 2
form2 = dot(nabla_grad(p), nabla_grad(q))*dx \
      - dot(nabla_grad(p_prev), nabla_grad(q))*dx \
      + (rho/dt)*div(u_sol)*q*dx
bilinear2 = lhs(form2)
linear2 = rhs(form2)

# step 3
form3 = rho*dot((u - u_sol), v)*dx \
      + dt*dot(nabla_grad(p_sol - p_prev), v)*dx
bilinear3 = lhs(form3)
linear3 = rhs(form3)

In [7]:
from dolfinx.fem.petsc import LinearProblem
from dolfinx.fem import assemble_scalar
from datetime import date

t = 0
i = 0
u_file = io.VTXWriter(domain.comm, f"{date.today()}/velocity.bp", u_sol)
p_file = io.VTXWriter(domain.comm, f"{date.today()}/pressure.bp", p_sol)
u_file.write(t)
p_file.write(t)

u_e = fem.Function(velocity_function_space)
u_e.interpolate(lambda x: np.vstack((4.0*x[1]*(1.0 - x[1]), 0.0*x[0])))
comm = u_sol.function_space.mesh.comm

with open(f"{date.today()}/error.txt", "w") as error_log:
    for n in range(num_steps):
        t += dt.value
        i += 1
        
        problem1 = LinearProblem(bilinear1, linear1, bc_u, u_sol)
        problem1.solve()
    
        problem2 = LinearProblem(bilinear2, linear2, bc_p, p_sol)
        problem2.solve()
    
        problem3 = LinearProblem(bilinear3, linear3, bc_u, u_sol)
        problem3.solve()
    
        u_file.write(t)
        p_file.write(t)
        
        if i % 10 == 0:
            # error relativo: |u_sol - u_analitica| / |u_analitica|
            # con norma L_inf
            #error = np.abs(u_e.x.array - u_sol.x.array).max()/np.abs(u_e.x.array).max()
    
            # con norma L_2
            error_abs_integral = fem.form(inner(u_e - u_sol, u_e - u_sol) * dx)
            error_abs = np.sqrt(comm.allreduce(assemble_scalar(error_abs_integral), op=MPI.SUM))
            norm_u_e_integral = fem.form(inner(u_e, u_e) * dx)
            norm_u_e = np.sqrt(comm.allreduce(assemble_scalar(norm_u_e_integral), op=MPI.SUM))
            error = error_abs / norm_u_e
            
            error_log.write('t = %.2f: error = %.3g' % (t, error) + "\n")
    
        u_prev.x.array[:] = u_sol.x.array
        p_prev.x.array[:] = p_sol.x.array

KeyboardInterrupt: 